In [1]:
import numpy as np

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [3]:
import math, copy, time

In [4]:
import matplotlib.pyplot as plt

In [5]:
class EncoderDecoder(nn.Module):
    """
        A standard Encoder-Decoder architecture.
        Base for this and many other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        """
            Take in and process masked src and target sequences.
        """
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [6]:
class Generator(nn.Module):
    """
        Define standard linear + softmax generation step.
    """
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [7]:
def clones(module, N):
    """
        Produce N identical layers.
    """
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [8]:
class Encoder(nn.Module):
    """
        Core encoder is a stack of N layers
    """
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        """
            Pass the input (and mask) through each layer in turn.
        """
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [9]:
class LayerNorm(nn.Module):
    """
        Construct a layernorm module (See citation for details).
    """
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [10]:
class SublayerConnection(nn.Module):
    """
        A residual connection followed by a layer norm.
        Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        """
            Apply residual connection to any sublayer with the same size.
        """
        return x + self.dropout(sublayer(self.norm(x)))

In [11]:
class EncoderLayer(nn.Module):
    """
        Encoder is made up of self-attn and feed forward (defined below)
    """
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [12]:
class Decoder(nn.Module):
    """
        Generic N layer decoder with masking.
    """
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [13]:
class DecoderLayer(nn.Module):
    """
        Decoder is made of self-attn, src-attn, and feed forward (defined below)
    """
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [14]:
def subsequent_mask(size):
    """
        Mask out subsequent positions.
    """
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

In [15]:
def attention(query, key, value, mask=None, dropout=None):
    """
        Compute 'Scaled Dot Product Attention'
    """
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [16]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        """
            Take in model size and number of heads.
        """
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [17]:
class PositionwiseFeedForward(nn.Module):
    """
        Implements FFN equation.
    """
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [18]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [19]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(x)

In [20]:
def make_model(src_vocab, tgt_vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1):
    """
        Helper: Construct a model from hyperparameters.
    """
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

In [21]:
class Batch:
    """
        Object for holding a batch of data with mask during training.
    """
    def __init__(self, src, trg=None, pad_input=0, pad_output=0):
        self.src = src
        self.src_mask = (src != pad_input).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = self.make_std_mask(self.trg, pad_output)
            self.ntokens = (self.trg_y != pad_output).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        """
            Create a mask to hide padding and future words.
        """
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask

In [22]:
def run_epoch(data_iter, model, loss_compute):
    """
        Standard Training and Logging Function
    """
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(batch.src, batch.trg, batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print("\tEpoch Step: %d Loss: %f Tokens per Sec: %f" % (i, loss / batch.ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
    return total_loss / total_tokens

In [23]:
global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    """
        Keep augmenting batch and calculate total number of tokens + padding.
    """
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)

In [24]:
class NoamOpt:
    """
        Optim wrapper that implements rate.
    """
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        """
            Update parameters and rate
        """
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        """
            Implement `lrate` above
        """
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))

In [25]:
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [26]:
class LabelSmoothing(nn.Module):
    """
        Implement label smoothing.
    """
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))

In [27]:
class SimpleLossCompute:
    """
        A simple loss compute and train function.
    """
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1)) / norm
        if self.opt is not None:
            loss.backward()
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.data.item() * norm

In [28]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.data[0]
        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
        if output_tokenizer.get_vocab['<end>'] == next_word:
            break
    return ys

In [29]:
from torchtext import data as data_

In [30]:
class MyIterator(data_.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))

In [31]:
def rebatch(null_input_token, null_output_token,  batch):
    "Fix order in torchtext to match ours"
    src, trg = x = batch['input'],  batch['output']
    return Batch(src, trg, null_input_token, null_output_token)

In [32]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from tokenizers import Tokenizer

In [33]:
from torch.utils.data import DataLoader

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
from tokenizers import Tokenizer, pre_tokenizers, normalizers, decoders, models, processors

In [36]:
from tokenizer import Tokenizer as Tokenize

In [37]:
from seq2seq_attn import (NLPDataset, collate_fn_padd) 

In [38]:
import pandas as pd

In [39]:
input_tokenizer = Tokenizer(models.BPE(
  vocab="./input_tokenizer-vocab.json", 
  merges="./input_tokenizer-merges.txt"
))

In [40]:
input_tokenizer.normalizer = normalizers.Lowercase()
input_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)
input_tokenizer.decoder = decoders.ByteLevel()
input_tokenizer.post_processor = BertProcessing(
  sep=("<end>", input_tokenizer.token_to_id("<end>")),
  cls=("<start>", input_tokenizer.token_to_id("<start>")),
)

In [41]:
special_tokens = ["<start>", "<pad>", "<end>", "<unk>", "<mask>"]

In [42]:
output_tokenizer = Tokenize(files='../data/output.txt', mode='word_level', checker='output',
                      special_tokens=special_tokens, max_length=10, tokenizer='./tokenizer_output.pickle')

In [43]:
null_input_token =  input_tokenizer.get_vocab()['<pad>']
null_output_token =  output_tokenizer.get_vocab['<pad>']
start_token = output_tokenizer.get_vocab['<start>']
end_token = output_tokenizer.get_vocab['<end>']

In [44]:
null_output_token

4464

In [45]:
data = pd.read_json('../data/final_df.json')
data.output = data.output.apply(lambda x: x.replace('/', ' '))

In [46]:
data = data.sample(len(data))

In [47]:
train = data[:int(0.999 * data.shape[0])]
val = data[int(0.999 * data.shape[0]):]

In [48]:
dataset_train = NLPDataset(train, input_tokenizer, output_tokenizer)
dataset_val = NLPDataset(val, input_tokenizer, output_tokenizer)

In [49]:
dataloader_train = DataLoader(dataset_train, batch_size=40, shuffle=True,
                              num_workers=3, collate_fn=collate_fn_padd)
dataloader_val = DataLoader(dataset_val, batch_size=40, shuffle=False, num_workers=3,
                            collate_fn=collate_fn_padd)

In [50]:
criterion = LabelSmoothing(size=len(output_tokenizer.get_vocab), padding_idx=null_output_token, smoothing=0.1)
model = make_model(len(input_tokenizer.get_vocab()), len(output_tokenizer.get_vocab), N=2)

/home/gabriel.pontes/.local/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [51]:
model_opt = NoamOpt(model.src_embed[0].d_model, 1, 2000,
                    torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [ ]:
for epoch in range(10):
    print("EPOCH: ", epoch)
    model.train()
    run_epoch((rebatch(null_input_token, null_output_token, b) for b in dataloader_train), 
                model, 
                SimpleLossCompute(model.generator, criterion, opt=model_opt))
    print()
        
    model.eval()
    loss = run_epoch((rebatch(null_input_token, null_output_token, b) for b in dataloader_val), 
                      model, 
                      SimpleLossCompute(model.generator, criterion, opt=None))
    print(loss)

EPOCH:  0


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


	Epoch Step: 1 Loss: 7.103647 Tokens per Sec: 134.426102
	Epoch Step: 51 Loss: 5.460472 Tokens per Sec: 251.403763
	Epoch Step: 101 Loss: 4.677882 Tokens per Sec: 252.397827
	Epoch Step: 151 Loss: 3.934355 Tokens per Sec: 249.568298
	Epoch Step: 201 Loss: 3.497052 Tokens per Sec: 237.112747
	Epoch Step: 251 Loss: 3.207328 Tokens per Sec: 251.349915
	Epoch Step: 301 Loss: 2.451070 Tokens per Sec: 251.747360
	Epoch Step: 351 Loss: 2.282195 Tokens per Sec: 254.836456
	Epoch Step: 401 Loss: 2.021716 Tokens per Sec: 253.108566
	Epoch Step: 451 Loss: 1.582152 Tokens per Sec: 254.119537
	Epoch Step: 501 Loss: 1.772688 Tokens per Sec: 249.432510
	Epoch Step: 551 Loss: 1.695111 Tokens per Sec: 255.712082
	Epoch Step: 601 Loss: 1.256668 Tokens per Sec: 252.056732
	Epoch Step: 651 Loss: 1.579850 Tokens per Sec: 251.320969
	Epoch Step: 701 Loss: 1.496598 Tokens per Sec: 240.520432
	Epoch Step: 751 Loss: 1.292640 Tokens per Sec: 255.256744
	Epoch Step: 801 Loss: 1.397658 Tokens per Sec: 254.983246


In [ ]:
torch.save(model, 'transformer.pt')

In [ ]:
def sample(x, y, model):
    model.eval()
    x = input_tokenizer.encode(x).ids
    x = torch.tensor(x).unsqueeze(0)
    
    #src = x.transpose(0, 1)[:1]
    src = x[:1]
    src_mask = (src != null_input_token).unsqueeze(-2)
    
    out = greedy_decode(model, src, src_mask, max_len=7, start_symbol=start_token)
    out = out[0]
    out = [item.data.item() for item in out]
    
    print("Translation:", end="\t")
    sym = output_tokenizer.decode(out)
    print(sym, end =" ")
    print()
    
    print("Target:", end="\t")
    tokens = output_tokenizer.encode(y).ids
    sym = output_tokenizer.decode(tokens)
    print(sym, end =" ")
    print()

In [ ]:
data.query_string.tolist()[1053], data.output.tolist()[1053]

In [ ]:
sample('tv led 15 polegadas', 'tv-e-home-theater tv tv-monitor', model)